In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from exp.nb_05 import *

# Get data

In [3]:
# Get data

x_train,y_train,x_valid,y_valid= get_data_normalized()

train_ds,valid_ds = Dataset(x_train, y_train),Dataset(x_valid, y_valid)
nh,bs = 50,512
c = y_train.max().item()+1
loss_func = F.cross_entropy


data = DataBunch(*get_dls(train_ds, valid_ds, bs), c)

In [4]:
bs,nh,c

(512, 50, 10)

# Rewrite code and use exception as code flow

In [13]:
class Parent():
    def __call__(self,cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): 
            print('Success')
            return True
        print('Fail')
        return False

class Child(Parent):
    def custom_func(self):
        print('Do some custom work')
        return True
    def begin_fit(self):
        print('Do something at begin fit')

In [15]:
c = Child()
c('custom_func')

Do some custom work
Success


True

In [ ]:
class Callback():
    _order=0
    def set_runner(self, run): self.run=run
    def __getattr__(self, k): return getattr(self.run, k)
    
    @property
    def name(self):
        name = re.sub(r'Callback$', '', self.__class__.__name__)
        return camel2snake(name or 'callback')
    
    # this part is somewhat identical to Runner __call__. We decide have it here as well
    # because you can create your own callback and add your own function (beside all the begin_fit, after_batch ...)
    # see example above
    def __call__(self, cb_name):
        f = getattr(self, cb_name, None)
        if f and f(): return True
        return False

class TrainEvalCallback(Callback):
    def begin_fit(self):
        self.run.n_epochs=0.
        self.run.n_iter=0
    
    def after_batch(self):
        if not self.in_train: return
        self.run.n_epochs += 1./self.iters
        self.run.n_iter   += 1
        
    def begin_epoch(self):
        self.run.n_epochs=self.epoch
        self.model.train()
        self.run.in_train=True

    def begin_validate(self):
        self.model.eval()
        self.run.in_train=False

class CancelTrainException(Exception): pass # to create same class as its parent but with different name
class CancelEpochException(Exception): pass
class CancelBatchException(Exception): pass